In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries installed!


In [41]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim
import geocoder
import folium
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


print('Libraries imported!')

Libraries imported!


In [3]:
data = requests.get("https://www.zumper.com/blog/mapped-san-francisco-neighborhood-rent-prices-summer-2019/").text

soup = BeautifulSoup(data, 'html.parser')
print(soup.prettify())


<!DOCTYPE html >
<!--[if IE 8]>    <html class="ie8" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-US">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://www.zumper.com/blog/xmlrpc.php" rel="pingback"/>
  <meta content="https://s3.amazonaws.com/zumpermedia/blog/wp-content/uploads/2019/06/13161344/SanFrancisco_MonthlyPriceMedianMap_Summer2019.png" property="og:image">
   <link href="https://www.zumper.com/blog/wp-content/uploads/2018/09/favicon.ico" rel="icon" type="image/png"/>
   <title>
    Mapped: San Francisco Neighborhood Rent Prices (Summer 2019)
   </title>
   <link data-minify="1" href="https://www.zumper.com/blog/wp-content/cache/min/1/b7f426d19fbe8382000f68cc4cf8040e.css" rel="stylesheet">
    <meta content="Check out San Francisco one-bedroom rents broken down by neighborhood on our Summer 2019 rent map 

In [4]:
nlist = []
plist = []


In [5]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        nlist.append(cells[0].text)
        plist.append(cells[1].text)
        

In [6]:
sf = pd.DataFrame({"Neighborhood": nlist,
                    "Price": plist
                           })

sf

,Neighborhood,Price
0,Alamo Square,"$3,600"
1,Bayview,"$2,550"
2,Bernal Heights,"$3,040"
3,Buena Vista/Ashbury Heights,"$3,600"
4,Castro,"$3,540"
5,Civic Center,"$3,380"
6,Cole Valley,"$3,000"
7,Cow Hollow,"$3,650"
8,Downtown,"$3,270"
9,Financial District,"$3,740"


Getting the list of SF neighborhoods from Wikipedia using BeatifulSoup HTML library.

In [7]:

data = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco").text

soup = BeautifulSoup(data, 'html.parser')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of neighborhoods in San Francisco - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"3aeb7dca-93c8-4753-a020-78a0046bbb99","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_San_Francisco","wgTitle":"List of neighborhoods in San Francisco","wgCurRevisionId":952030286,"wgRevisionId":952030286,"wgArticleId":1487556,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: archived copy as title","Webarchive template archiveis links","We

In [8]:
neighborhoodList = []

for row in soup.find_all( "div", class_="tocright")[0].findAll("a"):
    neighborhoodList.append(row.text)

sf_df = pd.DataFrame({"Neighborhood": neighborhoodList})

sf_df


,Neighborhood
0,1 Alamo Square
1,2 Anza Vista
2,3 Ashbury Heights
3,4 Balboa Park
4,5 Balboa Terrace
5,6 Bayview
6,7 Belden Place
7,8 Bernal Heights
8,9 Buena Vista
9,10 Butchertown (Old and New)


In [9]:
sf_df.shape

(123, 1)

In [10]:

def get_latlng(neighborhood):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Francisco, United States of America'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in sf_df["Neighborhood"].tolist() ]


In [12]:
coords

[[37.77722000000006, -122.43145999999996],
 [37.78048000000007, -122.44357999999994],
 [37.76467000000008, -122.44586999999996],
 [37.72493000000003, -122.44313999999997],
 [37.73180000000008, -122.46739999999994],
 [37.73345000000006, -122.38997999999998],
 [37.79090375067028, -122.40376034592448],
 [37.73903000000007, -122.41620999999998],
 [37.76888336163275, -122.43835164115626],
 [37.777120000000025, -122.41963999999996],
 [37.769952019569445, -122.43615601325229],
 [37.777120000000025, -122.41963999999996],
 [37.731567379827034, -122.4291655833967],
 [37.77609000000007, -122.39374999999995],
 [37.795390000000054, -122.40811999999994],
 [37.77863000000008, -122.41682999999995],
 [37.76161662329307, -122.449349133496],
 [37.76532000000003, -122.44986999999998],
 [37.763640000000066, -122.44038999999998],
 [37.79825000000005, -122.43572999999998],
 [37.71088000000003, -122.43769999999995],
 [37.77869000000004, -122.39333999999997],
 [37.74432000000007, -122.44270999999998],
 [37.759

In [13]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


sf_df['Latitude'] = df_coords['Latitude']
sf_df['Longitude'] = df_coords['Longitude']

print(sf_df.shape)
sf_df

(123, 3)


,Neighborhood,Latitude,Longitude
0,1 Alamo Square,37.777220,-122.431460
1,2 Anza Vista,37.780480,-122.443580
2,3 Ashbury Heights,37.764670,-122.445870
3,4 Balboa Park,37.724930,-122.443140
4,5 Balboa Terrace,37.731800,-122.467400
5,6 Bayview,37.733450,-122.389980
6,7 Belden Place,37.790904,-122.403760
7,8 Bernal Heights,37.739030,-122.416210
8,9 Buena Vista,37.768883,-122.438352
9,10 Butchertown (Old and New),37.777120,-122.419640


Getting the latitude and longitude coordinates of each SF neighborhood using Geocoder.

In [14]:

address = "San Francisco"

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco, United States of America {}, {}.'.format(latitude, longitude))

map_sf = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, neighborhood in zip(sf_df['Latitude'], sf_df['Longitude'], sf_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sf)  
    
map_sf

The geograpical coordinate of San Francisco, United States of America 37.7790262, -122.4199061.


In [15]:
sf_df.to_csv("sf_df.csv", index=False)
map_sf.save('map_sf.html')

Using the Foursquare api to find the top 10 most popular venues for each SF neighborhood.

In [19]:
CLIENT_ID = '55TVVEHXKDCY3HUM25GV5L2GNAHSHXUY4MAPDGC3D1MBZOXM' 
CLIENT_SECRET = '2BJL3ZN5V2WB5U540Z52IGZKSRJFDAKSGEVGSATD3KOXO4UD' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 55TVVEHXKDCY3HUM25GV5L2GNAHSHXUY4MAPDGC3D1MBZOXM
CLIENT_SECRET:2BJL3ZN5V2WB5U540Z52IGZKSRJFDAKSGEVGSATD3KOXO4UD


In [20]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sf_df['Latitude'], sf_df['Longitude'], sf_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
print()

(11980, 7)



In [22]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
1 Alamo Square,100,100,100,100,100,100
10 Butchertown (Old and New),100,100,100,100,100,100
100 South End,100,100,100,100,100,100
101 South of Market,100,100,100,100,100,100
102 South Park,100,100,100,100,100,100
103 Sunnydale,43,43,43,43,43,43
104 Sunnyside,100,100,100,100,100,100
105 Sunset District,100,100,100,100,100,100
106 Telegraph Hill,100,100,100,100,100,100


In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

venues_df['VenueCategory'].unique()[:50]

There are 324 uniques categories.


array(['Park', 'Historic Site', 'Spiritual Center', 'Dog Run',
       'Ice Cream Shop', 'Liquor Store', 'Sushi Restaurant',
       'German Restaurant', 'Pizza Place', 'Rock Club', 'Yoga Studio',
       'Bakery', 'Market', 'Bar', 'Seafood Restaurant', 'Coffee Shop',
       'New American Restaurant', 'Souvlaki Shop', 'BBQ Joint',
       'Gift Shop', 'Wine Bar', 'Southern / Soul Food Restaurant',
       'Pedestrian Plaza', 'Cocktail Bar', 'Bagel Shop', 'Event Space',
       'Italian Restaurant', 'Beer Garden', 'Butcher', 'Comic Shop',
       'Music Venue', 'American Restaurant', 'Toy / Game Store',
       'Bike Shop', 'Gastropub', 'Bubble Tea Shop', 'Restaurant',
       'French Restaurant', 'Mexican Restaurant', 'Bookstore', 'Spa',
       'Jazz Club', 'Grocery Store', 'Arts & Crafts Store', 'Bath House',
       'Opera House', 'Dance Studio', 'Concert Hall', 'Korean Restaurant',
       'Marijuana Dispensary'], dtype=object)

In [24]:

sf_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

sf_onehot['Neighborhoods'] = venues_df['Neighborhood'] 


fixed_columns = list(sf_onehot.columns[-3:]) + list(sf_onehot.columns[:-3])
sf_onehot = sf_onehot[fixed_columns]

print(sf_onehot.shape)
sf_onehot.head()

(11980, 325)


,Zoo,Zoo Exhibit,Neighborhoods,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Piercing Parlor,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Street 

In [25]:
sf_grouped = sf_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sf_grouped.shape)
sf_grouped

(123, 325)


,Neighborhoods,Zoo,Zoo Exhibit,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Piercing Parlor,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Street 

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


areaColumns = ['Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

neighborhoods_venues_sorted['Neighborhoods'] = sf_grouped['Neighborhoods']

for ind in np.arange(sf_grouped.shape[0]):
    row_categories = sf_grouped.iloc[ind, :].iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]


print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(123, 11)


,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1 Alamo Square,Coffee Shop,Pizza Place,New American Restaurant,Bakery,Sushi Restaurant,Ice Cream Shop,Cocktail Bar,Park,Wine Bar,Grocery Store
1,10 Butchertown (Old and New),Coffee Shop,Marijuana Dispensary,Pizza Place,Yoga Studio,Sushi Restaurant,Bakery,Wine Bar,Gym,Dance Studio,Sandwich Place
2,100 South End,Coffee Shop,Park,Gym,Pizza Place,New American Restaurant,Gym / Fitness Center,Spa,Yoga Studio,Liquor Store,Baseball Stadium
3,101 South of Market,Coffee Shop,Pizza Place,Marijuana Dispensary,Spa,Gym / Fitness Center,Café,Art Museum,Park,Gym,Japanese Restaurant
4,102 South Park,Coffee Shop,Gym,Park,Pizza Place,Gym / Fitness Center,Spa,Yoga Studio,Liquor Store,Bakery,Baseball Stadium
5,103 Sunnydale,Park,Café,Trail,Pizza Place,Sandwich Place,Convenience Store,Liquor Store,Light Rail Station,Golf Course,Bakery
6,104 Sunnyside,Park,Coffee Shop,Bakery,Sandwich Place,Mexican Restaurant,Pizza Place,Japanese Restaurant,Grocery Store,Yoga Studio,Bubble Tea Shop
7,105 Sunset District,Chinese Restaurant,Coffee Shop,Park,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Café,Grocery Store,Playground,Pizza Place
8,106 Telegraph Hill,Coffee Shop,Pizza Place,Italian Restaurant,New American Restaurant,Park,Seafood Restaurant,Wine Bar,Men's Store,Café,Scenic Lookout
9,107 Tenderloin,Coffee Shop,Marijuana Dispensary,Boutique,Hotel,Art Museum,Wine Bar,Bakery,Sandwich Place,Concert Hall,Sushi Restaurant


In [28]:

len(sf_grouped[sf_grouped["Restaurant"] > 0])



sf_restaurant = sf_grouped[["Neighborhoods","Restaurant"]]

sf_restaurant.head()

,Neighborhoods,Restaurant
0,1 Alamo Square,0.01
1,10 Butchertown (Old and New),0.01
2,100 South End,0.00
3,101 South of Market,0.01
4,102 South Park,0.00


Using kmeans clustering to find which cluster of neighborhoods have the most and least amount of restaurants.

In [32]:

kclusters = 3

sf_clustering = sf_restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_clustering)


kmeans.labels_[0:10]


sf_merged = sf_restaurant.copy()

# add clustering labels
sf_merged["Cluster Labels"] = kmeans.labels_

sf_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sf_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,1 Alamo Square,0.01,0
1,10 Butchertown (Old and New),0.01,0
2,100 South End,0.00,1
3,101 South of Market,0.01,0
4,102 South Park,0.00,1


In [33]:
sf_merged = sf_merged.join(sf_df.set_index("Neighborhood"), on="Neighborhood")

print(sf_merged.shape)
sf_merged.head() 

(123, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,1 Alamo Square,0.01,0,37.777220,-122.431460
1,10 Butchertown (Old and New),0.01,0,37.777120,-122.419640
2,100 South End,0.00,1,37.781514,-122.394055
3,101 South of Market,0.01,0,37.777570,-122.404350
4,102 South Park,0.00,1,37.781763,-122.394342


In [34]:
print(sf_merged.shape)
sf_merged.sort_values(["Cluster Labels"], inplace=True)
sf_merged

(123, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,1 Alamo Square,0.010000,0,37.777220,-122.431460
86,66 Midtown Terrace,0.010000,0,37.751200,-122.452550
85,65 Merced Manor,0.010000,0,37.732440,-122.481450
84,64 Merced Heights,0.010000,0,37.719640,-122.467630
80,60 Lower Haight,0.010000,0,37.772730,-122.433100
76,57 Little Russia,0.010000,0,37.719131,-122.432017
72,53 Lakeshore,0.010000,0,37.730983,-122.494299
71,52 Lakeside,0.010000,0,37.724330,-122.473840
69,50 Laguna Honda,0.010000,0,37.753942,-122.463868
68,5 Balboa Terrace,0.010000,0,37.731800,-122.467400


Using folium to create the map of SF neighborhood coordinates and the clusters.

In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
map_clusters.save('map_clusters.html')

In [37]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,1 Alamo Square,0.01,0,37.777220,-122.431460
86,66 Midtown Terrace,0.01,0,37.751200,-122.452550
85,65 Merced Manor,0.01,0,37.732440,-122.481450
84,64 Merced Heights,0.01,0,37.719640,-122.467630
80,60 Lower Haight,0.01,0,37.772730,-122.433100
76,57 Little Russia,0.01,0,37.719131,-122.432017
72,53 Lakeshore,0.01,0,37.730983,-122.494299
71,52 Lakeside,0.01,0,37.724330,-122.473840
69,50 Laguna Honda,0.01,0,37.753942,-122.463868
68,5 Balboa Terrace,0.01,0,37.731800,-122.467400


In [39]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
74,55 Lincoln Manor,0.0,1,37.802029,-122.457552
73,54 Laurel Heights,0.0,1,37.785760,-122.447230
52,35 Forest Knolls,0.0,1,37.756307,-122.454869
51,34 Forest Hill,0.0,1,37.748170,-122.464110
11,109 Twin Peaks,0.0,1,37.759915,-122.446982
103,81 Outer Sunset,0.0,1,37.753020,-122.503800
104,82 Pacific Heights,0.0,1,37.792910,-122.434640
10,108 Treasure Island,0.0,1,37.816667,-122.371578
16,113 Visitacion Valley,0.0,1,37.713070,-122.409310
50,33 Fisherman's Wharf,0.0,1,37.808935,-122.415907


In [40]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
82,62 Lower Nob Hill,0.020000,2,37.789440,-122.414540
59,41 India Basin,0.022727,2,37.736040,-122.370530
58,40 Hunters Point,0.017857,2,37.733215,-122.376969
13,110 Union Square,0.020000,2,37.801466,-122.401768
105,83 Parkmerced,0.020000,2,37.717220,-122.480310
115,92 Richmond District,0.020000,2,37.780660,-122.470890
45,29 Excelsior,0.020000,2,37.726127,-122.433226
28,13 Cayuga Terrace,0.020000,2,37.731567,-122.429166
9,107 Tenderloin,0.020000,2,37.785130,-122.414510
119,96 Sea Cliff,0.030000,2,37.788972,-122.486542


Most of the restaurants appear to be in the green cluster while the least amout of restaurants appear to be in the red cluster which is the best place to open up a new restaurant.